In [ ]:
#pip install psycopg2-binary

In [6]:
import pandas as pd
import psycopg2

In [88]:
DB="postgres"
USER="postgres"
PWD="Data608projectbigdata"
HOST="data608project-postgresql.cti8i5zrv6mp.us-east-1.rds.amazonaws.com"
PORT=5432

In [90]:
try:
    # Connect to the PostgreSQL database
    connection = psycopg2.connect(
        dbname=DB,
        user=USER,
        password=PWD,
        host=HOST,
        port=PORT
    )
    
    cursor = connection.cursor()
    
    cursor.execute("""
                DROP TABLE IF EXISTS "silver.consolidated_airline_reviews";
                """)
            
    connection.commit()
    print(f"Table 'silver.consolidated_airline_reviews' dropped successfully.")



    cursor = connection.cursor()
    
    # Create the users table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS "silver.consolidated_airline_reviews" (
    	airline_review_id SERIAL PRIMARY KEY,
        "airline_name" varchar(255) NOT NULL,
    	"rating_overall" float8 NULL,
    	"review_title" varchar(1000) NOT NULL,
    	"review_date" timestamp NOT NULL,
    	"verified" bool NULL,
    	"review_text" text NOT NULL,
    	"aircraft" varchar(255) NULL,
    	"type_of_traveller" varchar(255) NULL,
    	"seat_type" varchar(255) NULL,
    	"route" varchar(1000) NULL,
    	"flight_date" timestamp NULL,
    	"rating_seat_comfort" float8 NULL,
    	"rating_cabin_staff_service" float8 NULL,
    	"rating_food_and_beverages" float8 NULL,
    	"rating_ground_service" float8 NULL,
    	"rating_inflight_entertainment" float8 NULL,
    	"rating_wifi_and_connectivity" float8 NULL,
    	"rating_value_for_money" float8 NULL,
    	"recommended" bool NULL,
        created_by varchar(100) NOT NULL,
        created_date timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP,
        lastmodified_by varchar(100) NOT NULL,
        lastmodified_date timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP   
    );
    """)
    connection.commit()
    print("Table 'silver.consolidated_airline_reviews' created successfully.")

except (Exception, psycopg2.Error) as error:
    print("Error during drop-create operation:", error)
finally:
    if connection:
        cursor.close()
        connection.close()

Table 'silver.consolidated_airline_reviews' dropped successfully.
Table 'silver.consolidated_airline_reviews' created successfully.


In [93]:
try:
    # Connect to the PostgreSQL database
    connection = psycopg2.connect(
        dbname=DB,
        user=USER,
        password=PWD,
        host=HOST,
        port=PORT
    )

    cursor = connection.cursor()
    
    # Create the users table
    cursor.execute("""
        INSERT INTO "silver.consolidated_airline_reviews" 
        (
            airline_name,
        	rating_overall,
        	review_title,
        	review_date,
        	verified,
        	review_text,
            aircraft,
        	type_of_traveller,
        	seat_type,
        	route,
        	flight_date,
        	rating_seat_comfort,
        	rating_cabin_staff_service,
        	rating_food_and_beverages,
        	rating_ground_service,
        	rating_inflight_entertainment,
        	rating_wifi_and_connectivity,
        	rating_value_for_money,
        	recommended,
            created_by,
            created_date,
            lastmodified_by,
            lastmodified_date           
        )
        SELECT
            CAST("Airline Name" AS varchar(255)) AS airline_name,
        	CASE WHEN "Overall_Rating" = 'n' THEN NULL ELSE CAST("Overall_Rating" AS float8) END AS rating_overall,
        	CAST("Review_Title" AS varchar(1000)) AS review_title,
            CAST("Review_Date" AS timestamp) AS review_date,
        	CAST("Verified" AS bool) AS verified,
        	CAST("Review" AS text) AS review_text,
            CAST("Aircraft" AS varchar(255)) AS aircraft,
        	CAST("Type Of Traveller" AS varchar(255)) AS type_of_traveller,
        	CAST("Seat Type" AS varchar(255)) AS seat_type,
        	CAST("Route" AS varchar(1000)) AS route,
        	CAST("Flight_Date" AS timestamp) AS flight_date,
        	CAST("Seat Comfort" AS float8) AS rating_seat_comfort,
        	CAST("Cabin Staff Service" AS float8) AS rating_cabin_staff_service,
        	CAST("Food & Beverages" AS float8) AS rating_food_and_beverages,
        	CAST("Ground Service" AS float8) AS rating_ground_service,
        	CAST("Inflight Entertainment" AS float8) AS rating_inflight_entertainment,
        	CAST("Wifi & Connectivity" AS float8) AS rating_wifi_and_connectivity,
        	CAST("Value For Money" AS float8) AS rating_value_for_money,
        	CAST("Recommended" AS bool) AS recommended,
            'From_Kaggle' AS created_by,
            CURRENT_TIMESTAMP AS created_date,
            'From_Kaggle' AS lastmodified_by,
            CURRENT_TIMESTAMP AS lastmodified_date
        FROM "bronze.kaggle_airline_reviews"
    """)
    connection.commit()
    print('Inserted to "silver.consolidated_airline_reviews" successfully')

except (Exception, psycopg2.Error) as error:
    print("Error during insert operation:", error)
finally:
    if connection:
        cursor.close()
        connection.close()

Inserted to "silver.consolidated_airline_reviews" successfully


In [106]:
results = None
colnames = None

try:
    # Connect to the PostgreSQL database
    connection = psycopg2.connect(
        dbname=DB,
        user=USER,
        password=PWD,
        host=HOST,
        port=PORT
    )

    cursor = connection.cursor()
    
    query = 'SELECT * FROM "silver.consolidated_airline_reviews"'
     
    cursor.execute(query)
     
    results = cursor.fetchall()

    # Get the column names
    colnames = [desc[0] for desc in cursor.description]

    # Convert the results to a pandas DataFrame
    df = pd.DataFrame(results, columns=colnames)

except (Exception, psycopg2.Error) as error:
    print("Error during select operation:", error)
finally:
    if connection:
        cursor.close()
        connection.close()

print(df.head())
print(len(df))

   airline_review_id   airline_name  rating_overall  \
0                  1    AB Aviation             9.0   
1                  2    AB Aviation             1.0   
2                  3    AB Aviation             1.0   
3                  4  Adria Airways             1.0   
4                  5  Adria Airways             1.0   

                            review_title review_date  verified  \
0                "pretty decent airline"  2019-11-11      True   
1                   "Not a good airline"  2019-06-25      True   
2         "flight was fortunately short"  2019-06-25      True   
3    "I will never fly again with Adria"  2019-09-28     False   
4  "it ruined our last days of holidays"  2019-09-24      True   

                                         review_text       aircraft  \
0    Moroni to Moheli. Turned out to be a pretty ...           None   
1   Moroni to Anjouan. It is a very small airline...           E120   
2    Anjouan to Dzaoudzi. A very small airline an...  Embra

#### Upload Dataframe to Databricks volume

In [108]:
df.to_csv('data/silver.consolidated_airline_reviews.csv')

In [110]:
import requests

In [112]:
%run POC_DatabricksToken.ipynb

In [117]:
#Using Databricks Upload a file API
#https://docs.databricks.com/api/workspace/files/upload
databricks_instance = 'dbc-daf0509f-1d8b.cloud.databricks.com'
local_path = 'data/silver.consolidated_airline_reviews.csv'
remote_path = '/Volumes/workspace/data608/data608_volume/silver_consolidated_airline_reviews.csv'
headers = {'Authorization' : f"Bearer {DATABRICKS_TOKEN}"}
#print(headers)

with open(local_path, 'rb') as f:
    r = requests.put(
      f"https://{databricks_instance}/api/2.0/fs/files{remote_path}?overwrite=true",
      headers=headers,
      data=f)
    r.raise_for_status()

print(f"Uploaded '{local_path}' to '{remote_path}' on Databricks workspace {databricks_instance}")

Uploaded 'data/silver.consolidated_airline_reviews.csv' to '/Volumes/workspace/data608/data608_volume/silver_consolidated_airline_reviews.csv' on Databricks workspace dbc-daf0509f-1d8b.cloud.databricks.com
